# VOBS Dashboard (Geomagnetic Virtual Observatories)

Product documentation (check links at the right):  
https://www.space.dtu.dk/english/research/projects/project-descriptions/geomagnetic-virtual-observatories

In [ ]:
# !pip install --upgrade 'git+https://github.com/ESA-VirES/VirES-Python-Client@staging#egg=viresclient'

import os
import json
import datetime as dt
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from tqdm import tqdm
from viresclient import SwarmRequest

import panel as pn
pn.extension('ace')
import hvplot.xarray
import holoviews as hv
import geoviews as gv
hv.extension('bokeh')

# import warnings
# warnings.filterwarnings('ignore')

## Load in the GVO dataset

In [ ]:
import datetime as dt
import numpy as np
import xarray as xr
from tqdm import tqdm
from viresclient import SwarmRequest

SERVER_URL = 'https://vires.services/ows'

COLLECTIONS = {
    'Swarm_1M': 'SW_OPER_VOBS_1M_2_',
    'Swarm_4M': 'SW_OPER_VOBS_4M_2_',
    'Orsted_4M': 'OR_OPER_VOBS_4M_2_',
    'CHAMP_4M': 'CH_OPER_VOBS_4M_2_',
    'Cryosat_4M': 'CR_OPER_VOBS_4M_2_',
    'Composite_4M': 'CO_OPER_VOBS_4M_2_'
}
MODELS = {
    'CHAOS-Core': "'CHAOS-Core'(max_degree=14)",
    'MCO_SHA_2C': "'MCO_SHA_2C'(max_degree=14)",
    'MCO_SHA_2D': "'MCO_SHA_2D'(max_degree=14)",
    'IGRF': "IGRF"
}

def nec2rtp(ds):
    """Convert NEC coords to RTP. NB: data_var names do not change"""
    _ds = ds.copy()
    # Convert variables from NEC to RTP frame
    for var in _ds.data_vars:
        if "NEC" in _ds[var].dims:
            _ds[var] = np.array([-1, 1, -1])*_ds[var]
            _ds[var] = _ds[var].roll({"NEC": 1}, roll_coords=False)
            _ds[var].attrs = ds[var].attrs
    # Rename NEC dims & coords to RTP
    _ds = _ds.assign_coords({"NEC": ["Radial", "Theta", "Phi"]})
    _ds = _ds.rename_dims({"NEC": "RTP"}).rename({"NEC": "RTP"})
    _ds = _ds.set_index({"RTP": "RTP"}).set_coords("RTP")
    _ds["RTP"].attrs = {
        "units": "",
        "description": "RTP frame - Radial, Theta, Phi [R,T,P] = [-C,-N,E]"
    }
    return _ds

def fetch_vobs(collection, sv=False, reshape=True, rtp=True):
    """Fetch data from VirES for a given collection"""
    collection = f"{collection}:SecularVariation" if sv else collection
    if sv:
        measurements = ['SiteCode', 'B_SV', 'sigma_SV']
        models = []
    else:
        measurements = ['SiteCode', 'B_CF', 'B_OB', 'sigma_CF', 'sigma_OB']
        models = MODELS
    request = SwarmRequest(SERVER_URL)
    request.set_collection(collection)
    request.set_products(
        measurements=measurements,
        models=models
    )
    data = request.get_between(
        dt.datetime(2000, 1, 1),
        dt.datetime.now(),
        asynchronous=False, show_progress=False
    )
    ds = data.as_xarray(reshape=reshape)
    if rtp:
        ds = nec2rtp(ds)
    return ds

def merge_vobs(ds, ds_sv):
    """Merge in SecularVariation data by using new 'Timestamp_SV' coord"""
    ds_sv = ds_sv.rename({"Timestamp": "Timestamp_SV"})
    # Fill model data for SV as zero for now
    for model in MODELS.keys():
        ds_sv[f"B_NEC_{model}_SV"] = (
            ("SiteCode", "Timestamp_SV", "RTP"),
            np.zeros_like(ds_sv["B_SV"].values)
    )
    # Copy metadata
    attrs = ds.attrs.copy()
    for k, v in ds_sv.attrs.items():
        attrs[k].extend(v)
        attrs[k] = list(set(attrs[k]))
    ds = xr.merge((ds, ds_sv))
    ds.attrs = attrs
    return ds

def fetch_all_vobs():
    """Gives a dictionary containing datasets, one for each VOBS collection"""
    ALL_VOBS = {}
    for key, collection in tqdm(COLLECTIONS.items()):
        ds = fetch_vobs(collection)
        ds_sv = fetch_vobs(collection, sv=True)
        ds = merge_vobs(ds, ds_sv)
        ALL_VOBS[key] = ds
    return ALL_VOBS

# ALL_VOBS = fetch_all_vobs()
# print(ALL_VOBS["Swarm_1M"])
# # Now we can easily slice out data from specific GVOs:
# ALL_VOBS["Swarm_1M"].sel(RTP="Radial", Site=0)["B_OB"].plot.line(x="Timestamp")

In [ ]:
def data_fetching_code(cell=3):
    """Gives the Python code necessary to fetch & load the VOBS data"""
    filepath = os.path.join(os.path.abspath(''), "02_GVO-Geomagnetic-Virtual-Observatories.ipynb")
    with open(filepath) as f:
        r = json.load(f)
    return "".join(r["cells"][cell]["source"])

# print(data_fetching_code())

In [ ]:
# # Initialise the data in the cache
# if "ALL_VOBS" not in pn.state.cache:
#     ALL_VOBS = pn.state.cache["ALL_VOBS"] = fetch_all_vobs()
# else:
#     ALL_VOBS = pn.state.cache["ALL_VOBS"]

## Plot with cartopy/matplotlib

In [ ]:
LINE_COLORS = {
    "Radial": mpl.colors.to_hex("tab:blue"),
    "Theta": mpl.colors.to_hex("tab:orange"),
    "Phi": mpl.colors.to_hex("tab:green"),
    "Model": mpl.colors.to_hex("tab:grey"),
}

In [ ]:
def make_map(ds, RTP="Radial", highlight_Site=0):
    GVO_LOCATIONS = np.vstack((ds["Longitude"].values, ds["Latitude"].values)).T
    color = LINE_COLORS.get(RTP)
    fig = plt.figure(figsize=(8,4))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(),extent=[-180, 180, -90, 90])
    ax.coastlines()
    ax.scatter(*GVO_LOCATIONS.T, color="black", s=3, zorder=2)
    # Loop through each GVO
    for i in range(300):
        gvo_record = ds["B_OB"].sel(RTP=RTP).isel(SiteCode=i)
        # Extract a fractional (0-1) representation of the GVO data
#         gvo_y_scaled = (gvo_record.data - mini) / (maxi - mini)
        # SCALED SEPARATELY FOR EACH GVO
        gvo_y_scaled = (gvo_record.data - np.nanmin(gvo_record.data)) / (np.nanmax(gvo_record.data) - np.nanmin(gvo_record.data))
        # Construct vertices to represent the GVO data, scaled between -5 and 5
        # And centred around the GVO location in lat/lon space
        lat, lon = float(gvo_record.Latitude), float(gvo_record.Longitude)
        xdata = lon + np.linspace(-5, 5, len(gvo_y_scaled))
        ydata = lat + 10*(gvo_y_scaled - 0.5)
        gvo_xy_verts = np.vstack((xdata, ydata))
        # Plot these lines onto the figure
        ax.plot(*gvo_xy_verts, transform=ccrs.PlateCarree(), color=color, alpha=0.8, linewidth=0.8)
    # Draw a circle around selected grid point
    Site_latlon = ds[["Latitude", "Longitude"]].isel(SiteCode=highlight_Site)
    lat, lon = float(Site_latlon["Latitude"]), float(Site_latlon["Longitude"])
    ax.scatter(
        lon, lat, transform=ccrs.PlateCarree(),
        s=350, facecolors='grey', edgecolors='grey', linewidth=5
    )
    fig.text(0.5, 1, f"Selected LAT={round(lat, 1)}, LON={round(lon, 1)}", fontsize=15, ha="center", va="top")
    return fig

# make_map(ALL_VOBS["Swarm_1M"], RTP="Radial", highlight_Site=101);

## Redo map plot with GeoViews

In [ ]:
def wee_lines_for_map(ds, RTP="Radial", paths_or_points="points", param="OB"):
    """Returns GeoViews Paths for all the little lines to go on the map"""
    parameter = f"B_{param}"
    wee_paths = []
    lons = []
    lats = []
    for Site in range(300):
        gvo_record = ds[parameter].sel(RTP=RTP).isel(SiteCode=Site)
        if all(np.isnan(gvo_record.data)):
            continue
        # Extract a fractional (0-1) representation of the GVO data
        #         gvo_y_scaled = (gvo_record.data - mini) / (maxi - mini)
        # SCALED SEPARATELY FOR EACH GVO
        # subsamble the data to make it a bit lighter
        mini, maxi = np.nanmin(gvo_record.data), np.nanmax(gvo_record.data)
#         data = gvo_record.data[::4]
        data = gvo_record.data
        gvo_y_scaled = (data - mini) / (maxi - mini)
        # Screen out nans from this (they break the plotting)
        gvo_y_scaled = gvo_y_scaled[~np.isnan(gvo_y_scaled)]
        # Construct vertices to represent the GVO data, scaled between -5 and 5
        # And centred around the GVO location in lat/lon space
        lat, lon = float(gvo_record.Latitude), float(gvo_record.Longitude)
        xdata = lon + np.linspace(-5, 5, len(gvo_y_scaled))
        ydata = lat + 10*(gvo_y_scaled - 0.5)
        lons.append(xdata)
        lats.append(ydata)
    if paths_or_points=="points":
        # Merge all points from all GVOs to plot as one overlay
        lons = np.concatenate(lons)
        lats = np.concatenate(lats)
        lonlats = np.vstack((lons, lats)).T
        return gv.Overlay([gv.Points(lonlats, ["Longitude", "Latitude"]).opts(color=LINE_COLORS[RTP], size=0.5, alpha=1)], group="GVO_data")
    elif paths_or_points=="paths":
        # Plot data as paths, from each GVO individually (Slow)
        gvo_xy_verts_all = [dict(Longitude=xdata, Latitude=ydata) for xdata, ydata in zip(lons, lats)]
        wee_paths = [
            gv.Path(gvo_xy_verts).opts(color=LINE_COLORS[RTP])
            for gvo_xy_verts in gvo_xy_verts_all
        ]
        return gv.Overlay(wee_paths, group="GVO_data")


class GVOMap():
    
    def __init__(self, ds, RTP="Radial", draw_lines=True, use_pointer=True, param="OB"):
        self.GVO_LOCATIONS = np.vstack((ds["Longitude"].values, ds["Latitude"].values)).T
        self.basemap = gv.Overlay(self._basemap_with_GVO_points())
        if use_pointer:
            self.pointer_tap = hv.streams.Tap(source=self.basemap, x=0, y=90)
            self.basemap *= self._highlight_gvo()
        self.update_figure_with_lines(ds, RTP=RTP, draw_lines=draw_lines, param=param)
        # Set the pointer_tap as a property of the hv object
        # so we can easily extract current position with p.object.pointer_tap
        if use_pointer:
            self.object.pointer_tap = self.pointer_tap
    
    def update_figure_with_lines(self, ds, RTP="Radial", draw_lines=True, param="OB"):
        """Provides an updatable figure accessible at self.object"""
        if draw_lines:
            self.object = self.basemap * wee_lines_for_map(ds, RTP=RTP, param=param)
        else:
            self.object = self.basemap
        self.object.opts(
            projection=ccrs.PlateCarree(), aspect=2, global_extent=True, frame_width=500,
#             default_tools=[], tools=[], active_tools=[]  # why doesn't this work?
        )
    
    def _basemap_with_GVO_points(self):
        """Generates list of static GeoViews elements for the underlying map"""
        elements = [
            gv.feature.ocean.opts(alpha=0.7),
            gv.feature.land.opts(alpha=0.7),
        ]
        # Dots for the centres of all GVOs
        elements.append(
            gv.Points(self.GVO_LOCATIONS).opts(color="black", size=1)
#             .opts(default_tools=[], tools=[], active_tools=[])
        )
        return elements
    
    def _highlight_gvo(self):
        return gv.DynamicMap(
            lambda x, y: gv.Points([self.nearest_GVO(x, y)[:2]]),
            streams=[self.pointer_tap],
        ).opts(projection=ccrs.PlateCarree(), color="grey", size=18, alpha=0.8)

    def nearest_GVO(self, x, y):
        """Given (x, y) location, return (x, y, Site) of the nearest GVO"""
        Site = int(np.argmin(np.sum((self.GVO_LOCATIONS - np.array([x, y]))**2, axis=1)))
        x, y = self.GVO_LOCATIONS[Site]
        return x, y, Site
    
    def selected_GVO_Site(self):
        x, y = self.pointer_tap.x, self.pointer_tap.y
        _, _, Site = self.nearest_GVO(x, y)
        return Site

# p1 = GVOMap(ALL_VOBS["Swarm_1M"], draw_lines=True, use_pointer=True, RTP="Radial", param="OB")
# p1.update_figure_with_lines(ALL_VOBS["Swarm_1M"], "Theta", param="SV")
# p1.basemap + p1.object

## HoloViews plot for individual GVO

In [ ]:
class LineplotGVO:
    
    def __init__(self):
        pass
    
    @staticmethod
    def _select_obs_and_rename_vars(ds, Site, param, model):
        # Must split apart and rename components so that they are treated as
        # separate entities with shared_axes in hvplot
        # https://discourse.holoviz.org/t/how-to-only-link-share-only-the-x-axis/93
        # Select the data we want to access
        _ds = ds[[f"B_{param}", f"sigma_{param}", f"B_NEC_{model}"]]
        if param=="SV":
            _ds = _ds.rename({"Timestamp_SV": "Timestamp"})
        _ds = _ds.isel(SiteCode=Site)
        # Drop nans so that we get a continuous shaded area
#         _ds = _ds.dropna(dim="Timestamp")
        for RTP in ["Radial", "Theta", "Phi"]:
            _ds[f"B_{param} ({RTP})"] = _ds[f"B_{param}"].sel(RTP=RTP)
            _ds[f"sigma_{param} ({RTP})"] = _ds[f"sigma_{param}"].sel(RTP=RTP)
            _ds[f"B_{param} ({RTP}) - lower"] = _ds[f"B_{param}"].sel(RTP=RTP) - _ds[f"sigma_{param}"].sel(RTP=RTP)
            _ds[f"B_{param} ({RTP}) - upper"] = _ds[f"B_{param}"].sel(RTP=RTP) + _ds[f"sigma_{param}"].sel(RTP=RTP)
            _ds[f"B_NEC_{model} ({RTP})"] = _ds[f"B_NEC_{model}"].sel(RTP=RTP)
        _ds = _ds.drop_dims("RTP")
        return _ds

    def make_gvo_lineplot(self, ds=None, Site=0, param="OB", model="CHAOS-Core", fix_size=True):
        """Generate holoviews object showing the three components"""
        model = f"{model}_SV" if param=="SV" else model
        _ds = self._select_obs_and_rename_vars(ds, Site, param, model)
        color_r = LINE_COLORS["Radial"]
        color_t = LINE_COLORS["Theta"]
        color_p = LINE_COLORS["Phi"]
        color_model = LINE_COLORS["Model"]
        if fix_size:
            kwargs = dict(
                height=180,
                width=500
            )
        else:
            kwargs = {}
        lineplots = \
              hvplot.plot(_ds, kind="scatter", x="Timestamp", y=f"B_{param} (Radial)", line_color=color_r, marker="x", xlabel="", grid=True, **kwargs) \
                * _ds.hvplot.area(x="Timestamp", y=f"B_{param} (Radial) - lower", y2=f"B_{param} (Radial) - upper", color=color_r, alpha=0.5) \
                * hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_NEC_{model} (Radial)", line_color=color_model) \
            + hvplot.plot(_ds, kind="scatter", x="Timestamp", y=f"B_{param} (Theta)", line_color=color_t, marker="x", xlabel="", grid=True, **kwargs) \
                * _ds.hvplot.area(x="Timestamp", y=f"B_{param} (Theta) - lower", y2=f"B_{param} (Theta) - upper", color=color_t, alpha=0.5) \
                * hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_NEC_{model} (Theta)", line_color=color_model) \
            + hvplot.plot(_ds, kind="scatter", x="Timestamp", y=f"B_{param} (Phi)", line_color=color_p, marker="x", xlabel="", grid=True, **kwargs) \
                * _ds.hvplot.area(x="Timestamp", y=f"B_{param} (Phi) - lower", y2=f"B_{param} (Phi) - upper", color=color_p, alpha=0.5) \
                * hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_NEC_{model} (Phi)", line_color=color_model)
        title = f"Selected {_ds.SiteCode.values}; Radius {int(_ds.Radius.values/1e3):,}km"
        lineplots.cols(1).opts(shared_axes=True, sizing_mode="scale_both", title=title)
        return lineplots

# p = LineplotGVO().make_gvo_lineplot(ALL_VOBS["Swarm_1M"], Site=30, param="SV", model="CHAOS-Core")
# p

## Build dashboard with Panel

In [ ]:
class GVO_Dashboard:
    
    def __init__(self):
        pass       
        
    def initialise(self):
        self.load_data()
        self.panes = {
            "hv-map": pn.pane.HoloViews(),
            "hv-gvo-line": pn.pane.HoloViews(),
            "loading": self.loading_indicator,
        }
        self.widgets = {
            "select_series": pn.widgets.Select(name="Select time series:", options=list(COLLECTIONS.keys())),#, width=150),
            "select_component": pn.widgets.Select(name="Preview component on map:", options=["Radial", "Theta", "Phi"]),#, width=150),
            "select_param": pn.widgets.Select(name="Select parameter:", options=["OB", "CF", "SV"]),#, width=150),
            "select_model": pn.widgets.Select(name="Select model:", options=list(MODELS.keys())),#, width=150),
        }
        # Map with tap to select GVO
        self.hv_map = GVOMap(
            ds=self.ALL_VOBS[self.selected_series], RTP=self.selected_component,
            draw_lines=True, use_pointer=True, param=self.selected_param
        )
        self.panes["hv-map"].object = self.hv_map.object
        # Detailed line plots from GVOs
        self.panes["hv-gvo-line"].object = LineplotGVO().make_gvo_lineplot(
            ds=self.ALL_VOBS[self.selected_series],
            Site=self.selected_site, param=self.selected_param, model=self.selected_model
        )
        # Add watching of parameters to trigger figure updates
        self.hv_map.object.pointer_tap.param.watch(self.update_lineplot, "x")
        self.widgets["select_component"].param.watch(self.update_map, "value")
        self.widgets["select_param"].param.watch(self.update_map, "value")
        self.widgets["select_series"].param.watch(self.update_map, "value")
        self.widgets["select_model"].param.watch(self.update_map, "value")
    
    def load_data(self, force_update=False):
        """Load all used data from VirES"""
        # Load from global cache if available
        if ("ALL_VOBS" not in pn.state.cache) or force_update:
            pn.state.cache["ALL_VOBS"] = ALL_VOBS = fetch_all_vobs()
        else:
            ALL_VOBS = pn.state.cache["ALL_VOBS"]
        SOURCES = []
        for ds in ALL_VOBS.values():
            SOURCES.extend(ds.Sources)
        SOURCES = list(set(SOURCES))
        SOURCES.sort()
        self.ALL_VOBS = ALL_VOBS
        self.SOURCES = SOURCES
        
    @staticmethod
    @pn.depends(pn.state.param.busy)
    def loading_indicator(busy):
        if busy:
            return pn.indicators.Progress(visible=True, active=True, width=200, bar_color="primary")
        else:
            return ""

    @property
    def selected_series(self):
        return self.widgets["select_series"].value
    
    @property
    def selected_param(self):
        return self.widgets["select_param"].value
    
    @property
    def selected_component(self):
        return self.widgets["select_component"].value
    
    @property
    def selected_model(self):
        return self.widgets["select_model"].value
    
    @property
    def selected_site(self):
        return self.hv_map.selected_GVO_Site()
          
    def update_map(self, event):
        """Update the map to preview different time series"""
        self.hv_map.update_figure_with_lines(
            ds=self.ALL_VOBS[self.selected_series], RTP=self.selected_component,
            param=self.selected_param
        )
        self.panes["hv-map"].object = self.hv_map.object
        # Update the lineplots as well
        self.panes["hv-gvo-line"].object = LineplotGVO().make_gvo_lineplot(
            ds=self.ALL_VOBS[self.selected_series], Site=self.selected_site,
            param=self.selected_param, model=self.selected_model
        )

    def update_lineplot(self, event):
        """Update the line figure according to the pointer tap on the map"""
        x, y = event.obj.x, event.obj.y
        x, y, Site = self.hv_map.nearest_GVO(x, y)
        self.panes["hv-gvo-line"].object = LineplotGVO().make_gvo_lineplot(
            ds=self.ALL_VOBS[self.selected_series],
            Site=Site, param=self.selected_param, model=self.selected_model
        )

    def get_dashboard(self):
        # Create a temporary dashboard to display during loading
        # This is replaced by the real dashboard in on_load
        self.dashboard = pn.Column(
            pn.widgets.LoadingSpinner(value=True, align="center"),
            pn.pane.Markdown("""
                Assembling the dashboard... this will take about 1 minute  
                Check out the [project description](https://www.space.dtu.dk/english/research/projects/project-descriptions/geomagnetic-virtual-observatories) while you wait
                """,
                align="center",
                style={"text-align": "center"}
            ),
            sizing_mode="stretch_both"
        )
        def on_load():
            """Load the data and create the dashboard"""
            self.initialise()
            intro_text = pn.pane.Markdown("""
            Notes:

            - OB = observed total field; CF = core field estimate after denoising; SV: secular variation
            - grey lines: Model predictions (max_degree=14); NB: SV currently not calculated (and set to zero)
            - [Project documentation](https://www.space.dtu.dk/english/research/projects/project-descriptions/geomagnetic-virtual-observatories)
            """, dedent=True
            )
            sources_text = pn.pane.Markdown("Source files:  \n" + "  \n".join(self.SOURCES))
            self.dashboard.clear()
            self.dashboard.extend([
                pn.Row(
                    pn.Column(
                        intro_text,
                        pn.Row(self.widgets["select_series"], self.widgets["select_param"], self.widgets["select_model"], width=550),
                        "Click on the map to select a GVO:",
                        self.panes["hv-map"],
                        pn.Row(self.widgets["select_component"], self.panes["loading"]),  
                    ),
                    pn.Column(self.panes["hv-gvo-line"], width=400),
                ),
                sources_text,
#                 pn.pane.Markdown("Python code to fetch and load data using [viresclient](https://viresclient.readthedocs.io) & [xarray](https://xarray.pydata.org/):"),
#                 pn.widgets.Ace(value=data_fetching_code(), language="python", readonly=True, width=1000, height=500),
            ])
        pn.state.onload(on_load)
        return self.dashboard

In [ ]:
GVO_Dashboard().get_dashboard().servable(title="Geomagnetic Virtual Observatories")

---

TODO:
- rewrite LinePlotGVO to use jslink
- review caching mechanism; way to auto-renew each day if necessary
- extend eoxmagmod/VirES to provide SV directly?
- get ideas for advanced interactivity (e.g. fitting SH models)